In [ ]:
import csv
import requests
import time
import os

def download_package(package_name, save_dir):
    try:
        print(f"{package_name} 패키지 다운로드 시작...")
        # 패키지 정보 가져오기
        url = f'https://registry.npmjs.org/{package_name}'
        response = requests.get(url)
        if response.status_code != 200:
            print(f"{package_name} 정보 요청 실패: 상태 코드 {response.status_code}")
            return

        data = response.json()

        # 최신 버전의 tarball URL 가져오기
        latest_version = data['dist-tags']['latest']
        tarball_url = data['versions'][latest_version]['dist']['tarball']

        # 파일 다운로드
        tarball_response = requests.get(tarball_url)
        if tarball_response.status_code != 200:
            print(f"{package_name} tarball 다운로드 실패: 상태 코드 {tarball_response.status_code}")
            return

        # 저장 폴더가 없으면 생성
        os.makedirs(save_dir, exist_ok=True)

        filename = f"{package_name.replace('/', '_')}-{latest_version}.tgz"
        filepath = os.path.join(save_dir, filename)

        with open(filepath, 'wb') as f:
            f.write(tarball_response.content)

        print(f"{package_name} 패키지 다운로드 완료.")
    except Exception as e:
        print(f"{package_name} 패키지를 다운로드하는 중 오류 발생: {e}")

# 주피터 노트북에서는 os.getcwd()로 현재 작업 디렉토리를 가져옵니다.
script_dir = os.getcwd()

# 베이스 디렉토리를 스크립트 위치를 기준으로 설정합니다.
base_directory = os.path.abspath(os.path.join(script_dir, '..', 'data', 'NPM'))
neutral_directory = os.path.join(base_directory, 'neutral')

# CSV 파일 경로를 설정합니다.
csv_file_path = os.path.join(base_directory, 'npm_package_info.csv')

# 정상 패키지 목록 추출
normal_packages = []

with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        if row['threat_type'] == 'false_positive':
            package_name = row['name']
            if package_name:
                normal_packages.append(package_name)

print(f"총 {len(normal_packages)}개의 정상 패키지를 발견했습니다.")

# 패키지 다운로드
print("패키지 다운로드를 시작합니다...")
for idx, package in enumerate(normal_packages, start=1):
    print(f"{idx}/{len(normal_packages)}: {package} 다운로드 중...")
    download_package(package, neutral_directory)
    # API 요청 제한을 피하기 위해 딜레이 추가 (10초)
    time.sleep(10)
print("패키지 다운로드를 완료했습니다.")